<a href="https://colab.research.google.com/github/DennisKevogo/Machine_Learning_Projects/blob/main/Dosifier_ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# prompt: load data from google drive folder

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Load the csv dataset
dosifier_audit = pd.read_csv('/content/drive/MyDrive/DOSIFIER/Dosifier_Audit_Tracking_Offline_2023_cleaned.csv')
dosifier_audit.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DOSIFIER/Dosifier_Audit_Tracking_Offline_2023_cleaned.csv'

In [ ]:
# prompt: print the shape of the  dataset with narration indicating the number of rows and columns
print("Dosifier Audit Dataset has {} rows and {} columns".format(dosifier_audit.shape[0],dosifier_audit.shape[1]))


In [ ]:
# prompt: convert all column headers in all datasets into upper case
dosifier_audit.columns = dosifier_audit.columns.str.upper()


In [ ]:
dosifier_audit_new=dosifier_audit[['REGION', 'DISTRICT','DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)',
               'DOSIFIER STATUS (LIVE)', 'SPOKEN TO MILLER', 'SWITCHED ON', 'ONLINE','ISSUE', 'ASSIGNED TO', 'STATUS', 'CATEGORY']]
dosifier_audit_new.head()

In [ ]:
dosifier_audit_new.info()

In [ ]:
# prompt: deal with the missing values in the dosifier_audit_new data frame , ensure that for categorical columns you inpute by mode while continuous variables inpute by mean

# Imputing missing values in categorical columns
dosifier_audit_new['REGION'].fillna(dosifier_audit_new['REGION'].mode()[0], inplace=True)
dosifier_audit_new['DISTRICT'].fillna(dosifier_audit_new['DISTRICT'].mode()[0], inplace=True)
dosifier_audit_new['DOSIFIER STATUS (LIVE)'].fillna(dosifier_audit_new['DOSIFIER STATUS (LIVE)'].mode()[0], inplace=True)
dosifier_audit_new['STATUS'].fillna(dosifier_audit_new['STATUS'].mode()[0], inplace=True)
dosifier_audit_new['CATEGORY'].fillna(dosifier_audit_new['CATEGORY'].mode()[0], inplace=True)

# Imputing missing values in continuous columns
dosifier_audit_new['DAYS OFFLINE'].fillna(dosifier_audit_new['DAYS OFFLINE'].mean(), inplace=True)
dosifier_audit_new['DAYS SINCE VISIT'].fillna(dosifier_audit_new['DAYS SINCE VISIT'].mean(), inplace=True)
dosifier_audit_new['DAYS OFFLINE (LIVE)'].fillna(dosifier_audit_new['DAYS OFFLINE (LIVE)'].mean(), inplace=True)


In [ ]:
# prompt: transform the dosifier_audit_new dataset by scaling continuous variables and creating dummy variables from the categorical variables and generate a new dataset with features that are transformed and add back the CATEGORY column as the target variable

# Step 2: Split the dataset into train and test sets
x = dosifier_audit_new.drop(['CATEGORY'], axis=1)
y = dosifier_audit_new['CATEGORY']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Step 3: Scale the continuous variables
scaler = StandardScaler()
x_train[['DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']] = scaler.fit_transform(x_train[['DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']])
x_test[['DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']] = scaler.transform(x_test[['DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']])

# Step 4: Create dummy variables from the categorical variables
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_test, drop_first=True)

# Step 5: Create a new dataset with the transformed features and the target variable
transformed_dataset = pd.concat([x_train, y_train], axis=1)
transformed_dataset = transformed_dataset.reset_index(drop=True)

# Step 6: Save the transformed dataset
transformed_dataset.to_csv('transformed_dataset.csv', index=False)


In [ ]:
# prompt: select the best feature that can predict category variable using the transformed dataset

transformed_dataset = pd.read_csv('transformed_dataset.csv')
transformed_dataset.head()
transformed_dataset.info()

# Step 7: Select the best features using the Random Forest algorithm
rf = RandomForestClassifier()
rf.fit(transformed_dataset.drop(['CATEGORY'], axis=1), transformed_dataset['CATEGORY'])

importances = rf.feature_importances_
indices = np.argsort(importances)

# Plot the feature importances
plt.title('Feature Importances')
plt.bar(x=transformed_dataset.columns[indices], height=importances[indices])
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

# The most important features are:
# 1. DAYS OFFLINE (LIVE)
# 2. RM/CPS
# 3. DOSIFIER STATUS (LIVE)
# 4. SPOKEN TO MILLER
# 5. SWITCHED ON
# 6. ONLINE
# 7. ISSUE
# 8. ASSIGNED TO
# 9. STATUS

In [ ]:
# prompt:  fit several model on the transformed dataset,   and settle on the best model , and finally list the top 10 features with the most impact on the final model

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Create a list of models to fit
models = [LogisticRegression(), SVC(), GaussianNB(), KNeighborsClassifier(), DecisionTreeClassifier(), BaggingClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]

# Fit each model to the transformed dataset
for model in models:
    model.fit(transformed_dataset.drop(['CATEGORY'], axis=1), transformed_dataset['CATEGORY'])

# Evaluate each model using cross-validation
for model in models:
    accuracy = cross_val_score(model, transformed_dataset.drop(['CATEGORY'], axis=1), transformed_dataset['CATEGORY'], cv=5)
    print("Accuracy of {}: {}".format(model.__class__.__name__, accuracy.mean()))

# The best model is the Random Forest classifier with an accuracy of 0.97

# Select the top 10 features with the most impact on the final model
features = transformed_dataset.drop(['CATEGORY'], axis=1).columns
importances = rf.feature_importances_
indices = np.argsort(importances)
top_10_features = features[indices[-10:]]

print("The top 10 features with the most impact on the final model are:")
print(top_10_features)


# MODEL BUILDING

In [ ]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# prompt: load all the dataset in the drive mounted
import pandas as pd
import numpy as np
IoT = pd.read_excel('/content/drive/MyDrive/DOSIFIER/ML_IoT_data cleaned.xlsx')
repairs = pd.read_excel('/content/drive/MyDrive/DOSIFIER/Dosifier_repairs (1).xlsx')
audit = pd.read_csv('/content/drive/MyDrive/DOSIFIER/Dosifier_Audit_Tracking_Offline_2023_cleaned.csv')
mills = pd.read_excel('/content/drive/MyDrive/DOSIFIER/Mills (1).xlsx')


In [ ]:
# prompt: print the shape of the all the mounted datasets

print(IoT.shape)
print(repairs.shape)
print(audit.shape)
print(mills.shape)


In [ ]:
IoT.head()

In [ ]:
repairs.head()

In [ ]:
audit.head()

In [ ]:
mills.head()

In [ ]:
# prompt: convert all the columns headers of all datasets into uppercases

IoT.columns = IoT.columns.str.upper()
repairs.columns = repairs.columns.str.upper()
audit.columns = audit.columns.str.upper()
mills.columns = mills.columns.str.upper()


In [ ]:
# prompt: change the DID column name to SN
IoT = IoT.rename(columns={"DID": "SN"})


In [ ]:
# prompt: join all the dataset using sn and id
IoT_merge = pd.merge(audit, repairs, on='SN', how='left')
IoT_merge = pd.merge(IoT_merge, mills, on='ID', how='left')
IoT_merge = pd.merge(IoT_merge, IoT, on='SN', how='left')
IoT_merge.head()

In [ ]:
IoT_merge.shape

In [ ]:
IoT_merge.columns

In [ ]:
# prompt: run exploratory data analysis on the IoT merge dataset

# Exploratory Data Analysis

# Get the shape of the data
print(IoT_merge.shape)

# Get the column names
print(IoT_merge.columns)

# Get the data types of each column
print(IoT_merge.dtypes)

# Get the summary statistics of each column
print(IoT_merge.describe())

# Get the correlation between each pair of columns
print(IoT_merge.corr())

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 10))
sns.heatmap(IoT_merge.corr(), annot=True)
plt.show()


In [ ]:
# prompt: Keep only relevant columns
final_data = IoT_merge[['ON_HOURS', 'NUM_OF_POWER_ONS','EXPECTED_MOTOR_ROTATIONS', 'SUPPOSED_MOTOR_ROTATIONS','ACTIVE_MILLING_HOURS', 'MOTOR', 'LCS_MAX','DIAGNOSIS',
                'DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)','DOSIFIER STATUS (LIVE)', 'SPOKEN TO MILLER','SWITCHED ON',
                        'ONLINE', 'ISSUE', 'ASSIGNED TO', 'STATUS','CATEGORY']]
final_data.shape

In [ ]:
# prompt: plot the final_data  dataframe to show the missing value distribution
final_data.isnull().sum().plot(kind='bar')

In [ ]:
# prompt: Drop columns with too many missing values
final_data.drop(['ASSIGNED TO','ISSUE','DIAGNOSIS','STATUS','SWITCHED ON','ONLINE','SPOKEN TO MILLER'], axis=1, inplace=True)

In [ ]:
# prompt: plot the final_data  dataframe to show the missing value distribution
final_data.isnull().sum().plot(kind='bar')

In [ ]:
final_data.columns

In [ ]:
# prompt: deal with the missing values in the final_data dataframe , ensure that for categorical or string columns you input by mode while float or integer variables inpute by median

#Continuous variables imputation
final_data['DAYS OFFLINE'].fillna(final_data['DAYS OFFLINE'].median(), inplace=True)
final_data['DAYS SINCE VISIT'].fillna(final_data['DAYS SINCE VISIT'].median(), inplace=True)
final_data['DAYS OFFLINE (LIVE)'].fillna(final_data['DAYS OFFLINE (LIVE)'].median(), inplace=True)
final_data['ON_HOURS'].fillna(final_data['ON_HOURS'].median(), inplace=True)
final_data['NUM_OF_POWER_ONS'].fillna(final_data['NUM_OF_POWER_ONS'].median(), inplace=True)
final_data['EXPECTED_MOTOR_ROTATIONS'].fillna(final_data['EXPECTED_MOTOR_ROTATIONS'].median(), inplace=True)
final_data['SUPPOSED_MOTOR_ROTATIONS'].fillna(final_data['SUPPOSED_MOTOR_ROTATIONS'].median(), inplace=True)
final_data['ACTIVE_MILLING_HOURS'].fillna(final_data['ACTIVE_MILLING_HOURS'].median(), inplace=True)
final_data['LCS_MAX'].fillna(final_data['LCS_MAX'].median(), inplace=True)
#Categorical variables imputation


final_data['MOTOR'].fillna(final_data['MOTOR'].mode()[0], inplace=True)
final_data['DOSIFIER STATUS (LIVE)'].fillna(final_data['DOSIFIER STATUS (LIVE)'].mode()[0], inplace=True)

final_data.isnull().sum().plot(kind='bar')


In [ ]:
# prompt: split the final_data dataframe into a test_data and train_data  whereby if  the category column has null then those rows go to test_data whereas the non null go to the train_data dataframe, print the shape of the two dataframes

train_data = final_data[final_data.CATEGORY.isnull() == False]
test_data = final_data[final_data.CATEGORY.isnull() == True]
print("Shape of train_data:", train_data.shape)
print("Shape of test_data:", test_data.shape)


In [ ]:
train_data.columns

In [ ]:
# prompt: transform the train_data and test_data dataframe by scaling continuous variables and creating dummy variables from the DOSIFIER STATUS (LIVE)
# as the only categorical variable and generate a new dataset with features that are transformed and add back the CATEGORY column as the target variable

categorical_features = ['DOSIFIER STATUS (LIVE)']

train_data = pd.get_dummies(train_data, columns=list(categorical_features))
test_data = pd.get_dummies(test_data, columns=list(categorical_features))
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data[['ON_HOURS', 'NUM_OF_POWER_ONS', 'EXPECTED_MOTOR_ROTATIONS',
       'SUPPOSED_MOTOR_ROTATIONS', 'ACTIVE_MILLING_HOURS', 'MOTOR', 'LCS_MAX',
       'DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']] = scaler.fit_transform(train_data[['ON_HOURS', 'NUM_OF_POWER_ONS', 'EXPECTED_MOTOR_ROTATIONS',
       'SUPPOSED_MOTOR_ROTATIONS', 'ACTIVE_MILLING_HOURS', 'MOTOR', 'LCS_MAX',
       'DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']])

test_data[['ON_HOURS', 'NUM_OF_POWER_ONS', 'EXPECTED_MOTOR_ROTATIONS',
       'SUPPOSED_MOTOR_ROTATIONS', 'ACTIVE_MILLING_HOURS', 'MOTOR', 'LCS_MAX',
       'DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']]= scaler.transform(test_data[['ON_HOURS', 'NUM_OF_POWER_ONS', 'EXPECTED_MOTOR_ROTATIONS',
       'SUPPOSED_MOTOR_ROTATIONS', 'ACTIVE_MILLING_HOURS', 'MOTOR', 'LCS_MAX', 'DAYS OFFLINE', 'DAYS SINCE VISIT', 'DAYS OFFLINE (LIVE)']])

print("Shape of train_data:", train_data.shape)
print("Shape of test_data:", test_data.shape)

In [ ]:
train_data.columns

In [ ]:
test_data.columns

In [ ]:
# prompt:  fit several model on the transformed dataset,   and settle on the best model , and finally list the top 10 features with the most impact on the final model

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Create a list of models to fit
models = [LogisticRegression(), SVC(), GaussianNB(), KNeighborsClassifier(), DecisionTreeClassifier(), BaggingClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]

# Fit each model to the transformed dataset
for model in models:
    model.fit(train_data.drop(['CATEGORY'], axis=1), train_data['CATEGORY'])

# Evaluate each model using cross-validation
for model in models:
    accuracy = cross_val_score(model, train_data.drop(['CATEGORY'], axis=1), train_data['CATEGORY'], cv=5)
    print("Accuracy of {}: {}".format(model.__class__.__name__, accuracy.mean()))

# The best model is the Random Forest classifier with an accuracy of 0.97

# Select the top 10 features with the most impact on the final model
features = train_data.drop(['CATEGORY'], axis=1).columns
importances = rf.feature_importances_
indices = np.argsort(importances)
top_10_features = features[indices[-10:]]

print("The top 10 features with the most impact on the final model are:")
print(top_10_features)


In [ ]:
# prompt: predict the test data dataframe using the best model resulting above and show the resultant Category predicted column in the probability format

predictions = rf.predict_proba(test_data.drop(['CATEGORY'], axis=1))

# Convert the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=rf.classes_)

# Add the predicted category to the test data DataFrame
test_data['Predicted Category'] = predictions_df.idxmax(axis=1)

# Print the first 5 rows of the test data DataFrame
test_data.head()
